In [12]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from PIL import Image
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.optim import Adam
from tqdm import tqdm
from glob import glob
import torch.nn.functional as F
import time 
from torchvision import transforms
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import wandb
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision import models
device = "cuda" if torch.cuda.is_available() else "cpu"

In [34]:
class CoordConv2d(nn.Module):

    def __init__(self, device, in_channels, out_channels, kernel_size, padding, stride, input_size):
        super(CoordConv2d, self).__init__()
        self.device = device
        self.cc_xy = self.make_channels(input_size)
        
        
        self.conv = nn.Conv2d(in_channels+2, out_channels,
                              kernel_size=kernel_size, padding=padding, stride=stride)

    def make_channels(self, input_size):
        coord_vals = (2 * torch.arange(input_size) / input_size) - 1

        xchannel = coord_vals.repeat((input_size, 1)).unsqueeze(dim=0)

        ychannel = xchannel.permute(0, 2, 1)
       
        return torch.cat((xchannel.unsqueeze(dim=0), ychannel.unsqueeze(dim=0)), dim=1)

    def forward(self, x):
        n = x.shape[0]
        x = torch.cat((x, self.cc_xy.repeat(
            n, 1, 1, 1).to(self.device)), dim=1)
        
        print(x.shape)
        return self.conv(x)


In [35]:
tnsr = torch.randn(1,3,64,64).to("cpu")



In [37]:
conv1 = CoordConv2d(
                "cpu", 3, 3, kernel_size=1, padding=0, stride=1, input_size=64)




In [38]:
conv2 = nn.Conv2d(3, 3, kernel_size=1, padding=0, stride=1)

In [3]:


class BB_model(nn.Module):
    def __init__(self, device, coordConv=True):
        super().__init__()

        if coordConv:
            self.conv1 = CoordConv2d(
                device, 3, 3, kernel_size=1, padding=0, stride=1, input_size=64)
        else:
            self.conv1 = nn.Conv2d(3, 3, kernel_size=1, padding=0, stride=1)

        resnet = models.resnet34(weights=True)

        # for param in resnet.parameters():

        #     param.requires_grad = False

        layers = list(resnet.children())[:8]
        self.features1 = nn.Sequential(*layers[:6])
        self.features2 = nn.Sequential(*layers[6:])

        # self.fc2
        self.bb = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2),
        )

    def forward(self, x):

        x = self.conv1(x)

        x = self.features1(x)  # 1, 128, 32, 32

        x = self.features2(x)  # [1, 512, 8, 8]

        x = F.relu(x)

        x = nn.AdaptiveAvgPool2d((1, 1))(x)  # [ 1,512,1,1]

        x = x.view(x.shape[0], -1)  # [1, 512]

        return self.bb(x)


In [4]:
IMAGE_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/AllDatasetMix/images"
trn_df = pd.read_csv("/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/AllDatasetMix/trainAll.csv")
val_df = pd.read_csv("/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/AllDatasetMix/valAll.csv")

RESIZE_AMT = 64
BACTH_SIZE = 512

train_transforms =  A.Compose([
    A.Resize(width=RESIZE_AMT,height=RESIZE_AMT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(p=1)
])

val_transforms =  A.Compose([
    A.Resize(width=RESIZE_AMT,height=RESIZE_AMT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(p=1)
])


In [5]:

class CenterDataset(torch.utils.data.Dataset):
    def __init__(self,df,image_dir=IMAGE_DIR,transforms=None):
        self.image_dir = image_dir
        self.df = df
        self.image_ids = df.Image_Name.unique()
        self.transforms = transforms
        
    def __getitem__(self,ix):
        
        img_id = self.image_ids[ix]
        img_path = os.path.join(self.image_dir,img_id)
        
        img = cv2.imread(img_path)[:,:,::-1]
        
        data = self.df[self.df["Image_Name"]==img_id]
        
        
        x1 = data["X1"].values[0] * RESIZE_AMT
        y1 = data["Y1"].values[0] * RESIZE_AMT
        
        center_loc = torch.Tensor([x1,y1]).float()

        if self.transforms:
            transformed = self.transforms(image=img)
            
            image = transformed["image"]
            
    
        return image,center_loc
    def collate_fn(self,batch):
        return tuple(zip(*batch))
    
    def __len__(self):
        return len(self.image_ids)

In [6]:
train_ds = CenterDataset(trn_df,transforms=train_transforms)
test_ds = CenterDataset(val_df,transforms=val_transforms)

trainLoader = DataLoader(train_ds, batch_size=BACTH_SIZE,
	shuffle=True, num_workers=os.cpu_count(), pin_memory=True,drop_last=True)
testLoader = DataLoader(test_ds, batch_size=BACTH_SIZE,
	num_workers=os.cpu_count(), pin_memory=True,drop_last=True)



def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [7]:








def main_training(model, optimizer,scheduler, train_dl, test_dl, epochs,loss_fn):
    idx = 0

    prev_loss = 0
    
    
    epochTrainLoss = []
    epochValLoss = []
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0

        for x, y_bb in train_dl:
            batch = x.shape[0]
            x = x.cuda().float()

            y_bb = y_bb.cuda()

            
    
            out_bb = model(x)

            loss_bb = loss_fn(out_bb,y_bb).sum(1)
            # loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
            loss_bb = loss_bb.sum()

            loss = loss_bb

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            idx += 1

            total += batch

            sum_loss += loss.item()

        val_loss = val_epochs(model, test_dl,loss_fn)

        if i == 0:
            prev_loss = val_loss
        if val_loss < prev_loss:
            prev_loss = val_loss

            model_name = f"Regression_model_{str(prev_loss)}.pth"
            torch.save(model, model_name)

        train_loss = sum_loss/total
        epochTrainLoss.append(train_loss)
        epochValLoss.append(val_loss)
        
        scheduler.step(train_loss)

        # train_metrics = {"train/epoch": i+1, "train/train_loss": train_loss}

        # val_metrics = {"val/epoch": i+1, "val/val_loss": val_loss}
        # wandb.log({**train_metrics, **val_metrics})

        print(f"Epoch Number {i+1}")
        print("train_loss %.3f " % (train_loss))
        print("Validation Loss %.3f " % (val_loss))
        print("*"*8)
    
    return epochTrainLoss,epochValLoss
        


In [8]:
def val_epochs(model,val_loader,loss_fn):
    
    model.eval()
    total_val_loss = 0
    total = 0
    for x,y_bb in val_loader:
        
    
        x = x.cuda().float()
        y_bb = y_bb.cuda()
        
        out_bb = model(x)
        
        total += x.shape[0]
        with torch.no_grad():
            # loss_bb = F.l1_loss(out_bb,y_bb,reduction='none').sum(1)
            loss_bb = loss_fn(out_bb,y_bb).sum(1)
            loss_bb = loss_bb.sum()
            
            total_val_loss += loss_bb.item()
            
    return total_val_loss/total

In [9]:
n_epoch = 100
    
loss_fn = nn.L1Loss(reduction='none')

model = BB_model(device,coordConv=False).to(device)
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.006)
    
update_optimizer(optimizer, 0.001)
    
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    0.006,
    n_epoch,
    steps_per_epoch=len(train_ds)//BACTH_SIZE
)

    
trainLossModel2,valLossModel2=main_training(model=model,optimizer=optimizer,scheduler=scheduler,train_dl=trainLoader,test_dl=testLoader,epochs=n_epoch,loss_fn=loss_fn)
        

NameError: name 'device' is not defined

In [ ]:
epoch_x = np.arange(0, n_epoch, 1)


plt.ion()

plt.plot(epoch_x, trainLossModel2, label="Standard", lw=3)
plt.plot(epoch_x, trainLossModel1, label="CoordConv", lw=3)
plt.legend()
plt.title("Supervised Training Regression Error")
plt.xlabel("Epoch")
plt.ylabel("Error")